In [5]:
import shutil
from pathlib import Path
import os
os.environ["HF_HOME"] = "./cache/huggingface"
os.environ['TRANSFORMERS_CACHE'] = "./cache/huggingface/t_cache"
import torch
import whisper
from minlora import add_lora, LoRAParametrization, merge_lora
from whisper_finetune.utils import read_config

from whisper_finetune.model.lora import (
    disable_all_but_parametrized_grads,
)
from whisper.model import Linear as WLinear
from functools import partial
from ctranslate2.converters import TransformersConverter
from whisper_finetune.model.model_utils import save_model
from transformers.models.whisper.convert_openai_to_hf import (
    convert_openai_whisper_to_tfms,
)
from huggingface_hub import HfApi
from faster_whisper import WhisperModel

In [2]:
import wandb

def download_model_from_wandb(run_path: str, file_path: str, save_dir: str) -> str:
    """
    Download a model file from Weights & Biases and return the local file path.

    Parameters:
    - run_path: str. Path to the W&B run, e.g., "i4ds/whisper4sg/runs/28z8x0k4".
    - file_path: str. Path to the file in the W&B run, e.g., "40569234_output/last_model.pt".
    - save_dir: str. Local directory to save the file.

    Returns:
    - str: The local path to the downloaded file.
    """
    # Initialize W&B API
    api = wandb.Api()

    # Fetch the run
    run = api.run(run_path)

    # File save path
    save_path = f"{save_dir}/{file_path.split('/')[-1]}"

    # Download the file
    run.file(file_path).download(root=save_dir, replace=True)

    return save_path

# Example usage
run_path = "i4ds/whisper4sg/runs/zli3sf8y"
file_path = "52966296_output/last_model.pt"
save_dir = "./downloaded_models"
hu_model_path = "i4ds/whisper4sg-sg-lora"
merge_cp_dumb = save_dir+'/' +file_path.split('/')[0]+'/merged.pt'

model_local_path = download_model_from_wandb(run_path, file_path, save_dir)
print(model_local_path)

./downloaded_models/last_model.pt


In [6]:
whisper_model = whisper.load_model('large-v2', 'cpu')

In [7]:
config = read_config('configs/large-lora-decoder-mp-sg-corpus.yaml')

In [8]:
# Create LORA config
lora_config = {
    WLinear: {
        "weight": partial(LoRAParametrization.from_linear, **config["model"]["lora_config"]),
    },
}
if config['training']['train_only_decoder']:
    add_lora(whisper_model.decoder, lora_config=lora_config)
else:
    add_lora(whisper_model, lora_config=lora_config)
disable_all_but_parametrized_grads(whisper_model)

In [9]:
last_model = torch.load(f'{save_dir}/{file_path}')
whisper_model.load_state_dict(last_model['model_state_dict'])

<All keys matched successfully>

In [12]:
merge_lora(whisper_model)
save_model(whisper_model, merge_cp_dumb)

In [ ]:
hf_model_folder = Path(save_dir, 'hf_model')
hf_model_folder.mkdir(exist_ok=True)
ctranslate2_model_folder = Path('ct2_output')

# Convert to Huggingface Model
hf_model = convert_openai_whisper_to_tfms(merge_cp_dumb, hf_model_folder)

In [14]:
hf_model[0].save_pretrained(hf_model_folder)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'begin_suppress_tokens': [220, 50256]}


In [15]:
shutil.copyfile("cache/tokenizer.json", Path(hf_model_folder, "tokenizer.json"))
shutil.copyfile("cache/config.json", Path(hf_model_folder, "config.json"))

# Create readme
readme_content = f"""
# Model Information

This folder contains a converted model using ctranslate2.

## Wandb log
https://wandb.ai/{run_path}

## Files
- `tokenizer.json`: Tokenizer file.
- `config.json`: Configuration file.

## Conversion Details
The model was converted to ctranslate2 format with float16 quantization.

## Data
Model was trained on the full sg corpus, with part of the mozilla common voice 13.0 dataset and SRG data translated by faster-whisper-v2.
"""
with open(Path(hf_model_folder, "README.md"), 'w') as f:
    f.write(readme_content)

# Convert to ctranslate2
converter = TransformersConverter(
    hf_model_folder,
    copy_files=["tokenizer.json", "README.md"],
    load_as_float16=True 
)

converter.convert(output_dir=ctranslate2_model_folder, quantization="float16", force=True)

Loading checkpoint shards: 100%|██████████| 2/2 [04:28<00:00, 134.18s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PosixPath('ct2_output')

In [18]:
api = HfApi()
api.upload_folder(
    folder_path="ct2_output",
    repo_id=hu_model_path,
    repo_type='model',
)

model.bin: 100%|██████████| 3.09G/3.09G [03:49<00:00, 13.5MB/s] 


CommitInfo(commit_url='https://huggingface.co/i4ds/whisper4sg-sg-lora/commit/63775637746e5eae42ed8868abbff1c5dee4cfe3', commit_message='Upload folder using huggingface_hub', commit_description='', oid='63775637746e5eae42ed8868abbff1c5dee4cfe3', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
model = WhisperModel(hu_model_path, device="cuda", compute_type="float16")

config.json: 100%|██████████| 12.1k/12.1k [00:00<00:00, 566kB/s]

tokenizer.json: 100%|██████████| 2.48M/2.48M [00:00<00:00, 7.37MB/s]

vocabulary.json: 100%|██████████| 1.07M/1.07M [00:00<00:00, 5.73MB/s]
model.bin: 100%|██████████| 3.09G/3.09G [02:33<00:00, 20.1MB/s]


In [20]:
segments, info = model.transcribe("Schönleber_1.mp3", beam_size=5, language='de')

In [23]:
# to use pysubs2, the argument must be a segment list-of-dicts
results= []
for s in segments:
    segment_dict = {'start':s.start,'end':s.end,'text':s.text}
    results.append(segment_dict)

In [34]:
import pysubs2

subs = pysubs2.load_from_whisper(results)
subs.save('Schönleber_1.srt')